# Etapa 2: Extração e transformação para alcançar a massa de dados com indicadores relevantes no período de tempo entre 2010 a 2014 do dataset "EdStatsData.csv".

## 1. Importando libs

In [1]:
!pip3 install pandas
!pip3 install numpy

In [2]:
import pandas as pd
import numpy as np
import zipfile

## 2. Carregando o dataframe com os dados de todos os países e da faixa temporal.

In [3]:
zf = zipfile.ZipFile('../data/Edstats_csv.zip')
text_files = zf.infolist()
df_todos_paises = pd.DataFrame()
for csv_file in text_files:        
    if csv_file.filename == 'EdStatsData.csv':
        print("Abrindo o arquivo",csv_file.filename)
        df_todos_paises = pd.read_csv(zf.open(csv_file.filename))
        break

if df_todos_paises.empty:
    exit("Erro ao ler o arquivo 'EdStatsData.csv' que fica dentro do 'Edstats_csv.zip' na pasta 'data'.")
else:
    print("Arquivo 'EdStatsData.csv' carregado com sucesso.")

Abrindo o arquivo EdStatsData.csv
Arquivo 'EdStatsData.csv' carregado com sucesso.


## 3. Organizando nome de países definidos em "lista_suja_paises".

In [4]:
df_lista_suja_paises = pd.read_csv('../data/lista_suja_paises.csv', sep = ';')

df_lista_paises = df_lista_suja_paises.drop_duplicates()

df_lista_paises.index = range(df_lista_paises.shape[0])

df_lista_paises.to_csv('../data/lista_paises_aids.csv', index = False)

## 4. Extraindo e transformando dados referentes aos países definidos para "massa_bruta_pais_por_indicadores.csv".

Retira a última coluna do dataset "EdStatsData.csv" para manter apenas os dados úteis.

In [5]:
df_todos_paises_lista_colunas = df_todos_paises.columns.tolist()
ultima_coluna_lixo_massa = df_todos_paises_lista_colunas[len(df_todos_paises_lista_colunas)-1]
df_todos_paises = df_todos_paises.drop([ultima_coluna_lixo_massa], axis=1)

Seleciona apenas os registros dos países presentes em "lista_paises_aids.csv".

In [6]:
df_paises_filtrados = df_todos_paises.loc[df_todos_paises['Country Name'].isin(df_lista_paises['Country Name'])]

Escolhe os registros dos indicadores relevantes definidos em "codigos_indicadores_relevantes_aids.csv".

In [7]:
df_lista_indicadores = pd.read_csv('../data/codigos_indicadores_relevantes_aids.csv')
df_paises_filtrados = df_paises_filtrados.loc[df_paises_filtrados['Indicator Code'].isin(df_lista_indicadores['Indicator Code'])]

Recupera os anos a serem utilizados para selecionar a faixa temporal da massa final.

In [8]:
df_lista_anos = pd.read_csv('../data/lista_anos_aids.csv')
menor_ano = df_lista_anos.min()
maior_ano = df_lista_anos.max()

Aplica as restrições selecionadas anteriormente no dataset original e gera um conjunto de dados derivado para ser exportado em "massa_bruta_pais_por_indicadores_aids.csv".

In [9]:
for c in df_paises_filtrados.columns:
    if(df_paises_filtrados[c].dtype == np.float64):
        coluna_numerica = pd.to_numeric(df_paises_filtrados[c].name, errors='coerce')
        
        if(coluna_numerica < menor_ano[0] or coluna_numerica > maior_ano[0]):            
            del df_paises_filtrados[str(coluna_numerica)]

df_paises_filtrados.index = range(df_paises_filtrados.shape[0])

del df_paises_filtrados['Country Code']
del df_paises_filtrados['Indicator Name']

df_paises_filtrados = df_paises_filtrados.fillna(0)

df_paises_filtrados = df_paises_filtrados.round({'2010':2, '2011':2, '2012':2, '2013':2, '2014':2})

df_paises_filtrados.to_csv('../data/massa_bruta_pais_por_indicadores_aids.csv', index = False)